# Preparation

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution1D, AveragePooling1D
from tensorflow.keras.layers import concatenate
from sklearn.svm import SVR
import scipy.stats as stats

from tensorflow.keras.layers import Input, GRU, Bidirectional
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution1D, MaxPooling1D
from tensorflow.keras.layers import Multiply

In [ ]:
from google.colab import drive
drive_dir = '/content/drive'
drive.mount(drive_dir, force_remount=True)

# Data

In [ ]:
# needs to be uploaded from you device
CNNSVR_train_data = pd.read_csv("/content/cnn-svr_training_example.csv")
# print(CNNSVR_train_data.head())
CNN_SVR_test_data = pd.read_csv("/content/cnn-svr_testing_example.csv")
# print(CNN_SVR_test_data.head())
CNN_SVR_weight_path = "/content/cnn-svr_weights.h5"

CRNN_test_data = pd.read_csv("/content/input_example.csv")
# print(CRNN_test_data.head())
CRNN_weight_path = "/content/C_RNNCrispr_weights.h5"

# CNN-SVR model building


In [ ]:
# Model
def grna_preprocess(lines):
    length = 23
    data_n = len(lines)
    seq = np.zeros((data_n, length, 4), dtype=int)
    for l in range(data_n):
        data = lines[l]
        seq_temp = data
        for i in range(length):
            if seq_temp[i] in "Aa":
                seq[l, i, 0] = 1
            elif seq_temp[i] in "Cc":
                seq[l, i, 1] = 1
            elif seq_temp[i] in "Gg":
                seq[l, i, 2] = 1
            elif seq_temp[i] in "Tt":
                seq[l, i, 3] = 1
    return seq


def epi_preprocess(lines):
    length = 23
    data_n = len(lines)
    epi = np.zeros((data_n, length), dtype=int)
    for l in range(data_n):
        data = lines[l]
        epi_temp = data
        for i in range(length):
            if epi_temp[i] in "A":
                epi[l, i] = 1
            elif epi_temp[i] in "N":
                epi[l, i] = 0
    return epi


def preprocess(file_path, usecols):
    data = pd.read_csv(file_path, usecols=usecols)
    data = np.array(data)
    ctcf, dnase, h3k4me3, rrbs = epi_preprocess(data[:, 0]), epi_preprocess(data[:, 1]), epi_preprocess(data[:, 2]), epi_preprocess(data[:, 3])
    epi = []
    for i in range(len(data)):
        ctcf_t, dnase_t, h3k4me3_t, rrbs_t = pd.DataFrame(ctcf[i]), pd.DataFrame(dnase[i]), pd.DataFrame(h3k4me3[i]), pd.DataFrame(rrbs[i])
        epi_t = pd.concat([ctcf_t, dnase_t, h3k4me3_t, rrbs_t], axis=1)
        epi_t = np.array(epi_t)
        epi.append(epi_t)
    epi = np.array(epi)
    return epi


def load_data(train_file, test_file):
    train_data = pd.read_csv(train_file, usecols=[4, 9])
    train_data = np.array(train_data)
    train_seq, train_y = train_data[:, 0], train_data[:, 1]
    train_seq = grna_preprocess(train_seq)
    train_epi = preprocess(train_file, [5, 6, 7, 8])
    train_y = train_y.reshape(len(train_y), -1)

    test_data = pd.read_csv(test_file, usecols=[4, 9])
    test_data = np.array(test_data)
    test_seq, test_y = test_data[:, 0], test_data[:, 1]
    test_seq = grna_preprocess(test_seq)
    test_epi = preprocess(test_file, [5, 6, 7, 8])
    test_y = test_y.reshape(len(test_y), -1)
    return train_seq, test_seq, train_epi, test_epi, train_y, test_y


def build_model():
    tensor_input = Input(shape=(23, 8))
    seq_input, epi_input = tf.split(tensor_input, 2, axis=2)

    dropout = 0.3
    seq_conv1 = Convolution1D(256, 5, kernel_initializer='glorot_uniform', name='seq_conv_1')(seq_input)
    seq_act1 = Activation('relu', name='seq_activation1')(seq_conv1)
    seq_pool1 = AveragePooling1D(2, name='seq_pooling_1')(seq_act1)
    seq_drop1 = Dropout(dropout)(seq_pool1)

    seq_conv2 = Convolution1D(256, 5, kernel_initializer='glorot_uniform', name='seq_conv_2')(seq_drop1)
    seq_act2 = Activation('relu', name='seq_activation_2')(seq_conv2)
    seq_pool2 = AveragePooling1D(2, name='seq_pooling_2')(seq_act2)
    seq_drop2 = Dropout(dropout)(seq_pool2)
    seq_flat = Flatten()(seq_drop2)

    seq_dense1 = Dense(256, activation='relu', name='seq_dense_1')(seq_flat)
    seq_drop3 = Dropout(dropout)(seq_dense1)
    seq_dense2 = Dense(128, activation='relu', name='seq_dense_2')(seq_drop3)
    seq_drop4 = Dropout(dropout)(seq_dense2)
    seq_dense3 = Dense(64, activation='relu', name='seq_dense_3')(seq_drop4)
    seq_drop5 = Dropout(dropout)(seq_dense3)
    seq_out = Dense(40, activation='relu', name='seq_dense_4')(seq_drop5)

    epi_conv1 = Convolution1D(256, 5, kernel_initializer='glorot_uniform', name='epi_conv_1')(epi_input)
    epi_act1 = Activation('relu', name='epi_activation_1')(epi_conv1)
    epi_pool1 = AveragePooling1D(2, name='epi_pooling_1')(epi_act1)
    epi_drop1 = Dropout(dropout)(epi_pool1)

    epi_conv2 = Convolution1D(256, 5, kernel_initializer='glorot_uniform', name='epi_conv_2')(epi_drop1)
    epi_act2 = Activation('relu', name='epi_activation_2')(epi_conv2)
    epi_pool2 = AveragePooling1D(2, name='epi_pooling_2')(epi_act2)
    epi_drop2 = Dropout(dropout)(epi_pool2)
    epi_flat = Flatten()(epi_drop2)

    epi_dense1 = Dense(256, activation='relu', name='epi_dense_1')(epi_flat)
    epi_drop3 = Dropout(dropout)(epi_dense1)
    epi_dense2 = Dense(128, activation='relu', name='epi_dense_2')(epi_drop3)
    epi_drop4 = Dropout(dropout)(epi_dense2)
    epi_dense3 = Dense(64, activation='relu', name='epi_dense_3')(epi_drop4)
    epi_drop5 = Dropout(dropout)(epi_dense3)
    epi_out = Dense(40, activation='relu', name='epi_dense_4')(epi_drop5)

    merged = concatenate([seq_out, epi_out], axis=-1)

    model = Model(inputs=tensor_input, outputs=[merged])

    # # Load weights for the model
    # print("Loading weights for the models")
    # model.load_weights(weight_path, by_name=True)

    # prediction = Dense(1, activation='linear', name='prediction')(merged)
    # final_model = Model([seq_input, epi_input], prediction)
    return model

Try out CNN-SVR

In [ ]:
# Load weights for the model
print("Loading weights for the models")
model = build_model()
model.load_weights(weight_path, by_name = True)

# Load training and testing data
print("Loading training and testing data")
seq_train, seq_test, epi_train, epi_test, y_train, y_test = load_data(train_data_path, test_data_path)
input_train = tf.concat([seq_train, epi_train], 2)
input_test = tf.concat([seq_test, epi_test], 2)

# Training and testing data shape
print("training sequence data shape: " + str(seq_train.shape))
print("training epigenetic data shape: " + str(epi_train.shape))
print("training combined data shape: " + str(input_train.shape))

print("testing sequence data shape: " + str(seq_test.shape))
print("testing epigenetic data shape: " + str(epi_test.shape))
print("testing combined data shape: " + str(input_test.shape))


# Predict on data
print("Predicting on training and testing data")
x_train = model.predict(input_train)
x_test = model.predict(input_test)

print("output (training) shape: " + str(x_train.shape))
print("output (testing) shape: " + str(x_test.shape))

x_train, x_test = np.array(x_train), np.array(x_test)

# Select important features from outputs of the CNN model
selected_cnn_fea_cols = [17, 26, 9, 19, 30, 6, 12, 39, 36, 21, 22, 3, 25]
x_train = x_train[:, selected_cnn_fea_cols]
x_test = x_test[:, selected_cnn_fea_cols]

print("training data shape: " + str(x_train.shape))
print("testing data shape: " + str(x_test.shape))

y_train = np.array(y_train).ravel()
y_test = np.array(y_test).ravel()

# SVR model
clf = SVR(kernel="rbf", gamma=0.12, C=1.7, epsilon=0.11, verbose=1)
# Fit the SVR model according to the given training data
clf.fit(x_train, y_train)

# Perform regression on samples in x_train
y_pred_train = clf.predict(x_train)
# Perform regression on samples in x_test
y_pred = clf.predict(x_test)

# Output from SVR
print("prediction: " + str(y_pred))
print("target: " + str(y_test))

# CRNNCrispr model building


In [ ]:
# Model
def grna_preprocess(lines):
    length = 23
    data_n = len(lines)
    seq = np.zeros((data_n, length, 4), dtype=int)
    for l in range(data_n):
        data = lines[l]
        seq_temp = data
        for i in range(length):
            if seq_temp[i] in "Aa":
                seq[l, i, 0] = 1
            elif seq_temp[i] in "Cc":
                seq[l, i, 1] = 1
            elif seq_temp[i] in "Gg":
                seq[l, i, 2] = 1
            elif seq_temp[i] in "Tt":
                seq[l, i, 3] = 1
    return seq


def epi_preprocess(lines):
    length = 23
    data_n = len(lines)
    epi = np.zeros((data_n, length), dtype=int)
    for l in range(data_n):
        data = lines[l]
        epi_temp = data
        for i in range(length):
            if epi_temp[i] in "A":
                epi[l, i] = 1
            elif epi_temp[i] in "N":
                epi[l, i] = 0
    return epi


def preprocess(file_path, usecols):
    data = pd.read_csv(file_path, usecols=usecols)
    data = np.array(data)
    epi_1, epi_2, epi_3, epi_4 = epi_preprocess(data[:, 0]), epi_preprocess(data[:, 1]), epi_preprocess(data[:, 2]), epi_preprocess(data[:, 3])
    epi = []
    for i in range(len(data)):
        epi_1_temp, epi_2_temp, epi_3_temp, epi_4_temp = pd.DataFrame(epi_1[i]), pd.DataFrame(epi_2[i]), pd.DataFrame(
            epi_3[i]), pd.DataFrame(epi_4[i])
        epi_temp = pd.concat([epi_1_temp, epi_2_temp, epi_3_temp, epi_4_temp], axis=1)
        epi_temp = np.array(epi_temp)
        epi.append(epi_temp)
    epi = np.array(epi)
    return epi


def load_data(test_file):
    test_data = pd.read_csv(test_file, usecols=[4, 9])
    test_data = np.array(test_data)
    x_test, y_test = test_data[:, 0], test_data[:, 1]
    x_test = grna_preprocess(x_test)
    epi_test = preprocess(test_file, [5, 6, 7, 8])
    y_test = y_test.reshape(len(y_test), -1)
    return x_test, epi_test, y_test

def build_CRNNCrispr():
    tensor_input = Input(shape=(23, 8))
    seq_input, epi_input = tf.split(tensor_input, 2, axis=2)

    seq_conv1 = Convolution1D(256, 5, kernel_initializer='random_uniform', name='seq_conv1')(seq_input)
    seq_act1 = Activation('relu')(seq_conv1)
    seq_pool1 = MaxPooling1D(2)(seq_act1)
    seq_drop1 = Dropout(0.2)(seq_pool1)
    gru1 = Bidirectional(GRU(256, kernel_initializer='he_normal', dropout=0.3, recurrent_dropout=0.2, reset_after=False), name='gru1')(seq_drop1)
    seq_dense1 = Dense(256, name='seq_dense1')(gru1)
    seq_act2 = Activation('relu')(seq_dense1)
    seq_drop2 = Dropout(0.3)(seq_act2)
    seq_dense2 = Dense(128, name='seq_dense2')(seq_drop2)
    seq_act3 = Activation('relu')(seq_dense2)
    seq_drop3 = Dropout(0.2)(seq_act3)
    seq_dense3 = Dense(64, name='seq_dense3')(seq_drop3)
    seq_act4 = Activation('relu')(seq_dense3)
    seq_drop4 = Dropout(0.2)(seq_act4)
    seq_dense4 = Dense(40, name='seq_dense4')(seq_drop4)
    seq_act5 = Activation('relu')(seq_dense4)
    seq_drop5 = Dropout(0.2)(seq_act5)

    epi_conv1 = Convolution1D(256, 5, name='epi_conv1')(epi_input)
    epi_act1 = Activation('relu')(epi_conv1)
    epi_pool1 = MaxPooling1D(2)(epi_act1)
    epi_drop1 = Dropout(0.3)(epi_pool1)
    epi_dense1 = Dense(256, name='epi_dense1')(epi_drop1)
    epi_act2 = Activation('relu')(epi_dense1)
    epi_drop2 = Dropout(0.2)(epi_act2)
    epi_dense2 = Dense(128, name='epi_dense2')(epi_drop2)
    epi_act3 = Activation('relu')(epi_dense2)
    epi_drop3 = Dropout(0.3)(epi_act3)
    epi_dense3 = Dense(64, name='epi_dense3')(epi_drop3)
    epi_act4 = Activation('relu')(epi_dense3)
    epi_drop4 = Dropout(0.3)(epi_act4)
    epi_act5 = Dense(40, name='epi_dense4')(epi_drop4)
    epi_out = Activation('relu')(epi_act5)

    seq_epi_m = Multiply()([seq_drop5, epi_out])
    seq_epi_drop = Dropout(0.2)(seq_epi_m)
    seq_epi_flat = Flatten()(seq_epi_drop)
    seq_epi_output = Dense(1, activation='linear')(seq_epi_flat)

    return Model(inputs=tensor_input, outputs=[seq_epi_output])

In [ ]:
# Load and try out CRNNCrispr
print("Loading weights for the models")
CRNNCrispr_model = build_CRNNCrispr()
CRNNCrispr_model.load_weights(weight_path) # needs to upload

print("Loading test data")
x_test, epi_test, y_test = load_data(data_path) # needs to upload

print("input sequence data shape: " + str(x_test.shape))
print("input epigenetic data shape: " + str(epi_test.shape))
input_test = tf.concat([x_test, epi_test], 2)

print("Predicting on test data")
y_pred = CRNNCrispr_model.predict(input_test, batch_size=256, verbose=2)
print("output prediction shape: " + str(y_pred.shape))
print("output target shape: " + str(y_test.shape))

# result = pd.concat([y_test, y_pred], axis=1)
# result.to_csv(result_file, index=False, sep=',', header=['y_test', 'y_pred'])

# Saliency Map

In [ ]:
def count_nonzero(saliency_map):
    count = 0
    nonzero = []
    for i in range(len(saliency_map)):
        if np.any(saliency_map[i]):
            count += 1
            nonzero.append(i)
    return count, nonzero


def count_nonzero_grad(saliency_map):
    count = 0
    for i in saliency_map:
        if np.any(i):
            count += 1
    return count


def find_max(saliency_map):
    max = 0
    index = 0
    for i in range(len(saliency_map)):
        cur = np.sum(np.abs(np.array(saliency_map[i]).ravel()))
        if cur > max:
            max = cur
            index = i
    return max, index

In [ ]:
def generate_saliency_map(input, model, target_layer_idx=-1):
    # Finds the target layer
    target_layer = Model(inputs=model.input, outputs=model.layers[target_layer_idx].output)
    tensor_input = tf.cast(input, tf.float32)

    with tf.GradientTape(persistent = True) as tape:
        tape.watch(tensor_input)
        output = target_layer(tensor_input)

    # grads = [tape.gradient(output[:, i], [seq, epi]) for i in range(output.shape[1])]
    jacob = tape.batch_jacobian(output, tensor_input)

    # Normalize the gradients
    print("Dimension original: " + str(jacob.shape))
    jacob /= (tf.reduce_max(tf.abs(jacob), axis = (2,3), keepdims = True) + 1e-8)
    print("Dimension after: " + str(jacob.shape))

    # Clean up the persistent gradient tape
    del tape

    return jacob


CNNSVR_saliency_map = generate_saliency_map(input_train[:100], model)

CNNSVR_model_output = model(input_train[:100])

print("Model outputs shape: " + str(CNNSVR_model_output.shape))
print("Input data saliency map shape: " + str(CNNSVR_saliency_map.shape))

In [ ]:
print(str(count_nonzero(CNNSVR_model_output)[0]) + " out of " + str(len(model_output))
+ " samples in the CNNSVR model outputs are nonzero")
print(str(count_nonzero(CNNSVR_saliency_map)[0]) + " out of " + str(len(saliency_map))
+ " samples in the CNNSVR saliency maps are nonzero")
print(str(count_nonzero(CNNSVR_saliency_map)[1]))

In [ ]:
_, max_index = find_max(CNNSVR_saliency_map)
print(max_index)

In [ ]:
num_map, map_list = count_nonzero(CNNSVR_saliency_map[max_index])

print(num_map)
print(map_list)

In [ ]:
def draw_saliency_map(saliency_map,):
    # Normalize the saliency map
    saliency_map /= (tf.reduce_max(tf.abs(saliency_map)) + 1e-8)

    # Transpose the map to match the format in the paper
    saliency_map = np.transpose(saliency_map)

    # Create a heatmap by applying a colormap (e.g., 'viridis')
    heatmap = plt.get_cmap('viridis')(saliency_map)

    # Display the original image, saliency map, and overlaid image
    plt.figure(figsize=(12, 6))
    plt.imshow(saliency_map, cmap='viridis')
    plt.colorbar()

    plt.title('Saliency Map of CNN-SVR for max data that is nonzero')
    y_ticks = np.arange(0, 8, 1)  # Custom tick locations
    y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

    x_ticks = np.arange(0, 23, 1)  # Custom tick locations
    x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

    plt.xticks(x_ticks, x_tick_labels)
    plt.yticks(y_ticks, y_tick_labels)

    plt.xlabel('Genomic Position')
    plt.ylabel('Nucleotide')

    plt.show()

In [ ]:
for i in map_list:
  draw_saliency_map(CNNSVR_saliency_map[max_index][i])

In [ ]:
# Saliency Map for CRNNCrispr
def generate_saliency_map(input, model, target_layer_idx=-1):
    # Finds the target layer
    target_layer = Model(inputs=model.input, outputs=model.layers[target_layer_idx].output)

    # Compute the gradients of the target class with respect to the model's output
    tensor_input = tf.convert_to_tensor(input, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(tensor_input)
        output = target_layer(tensor_input)
    grads = tape.gradient(output, tensor_input)

    # Normalize the gradients
    grads /= (tf.reduce_max(tf.abs(grads), axis=(1,2), keepdims=True) + 1e-8)

    # Create a saliency map by averaging the absolute gradients across color channels
    # saliency_map = np.mean(np.abs(grads[0]), axis=-1)
    saliency_map = grads

    return saliency_map

data_path = "/content/x_input_reg.npy"
# seq, epi, label = load_data(data_path)
# concat_input = tf.concat([seq, epi], 2)
# concat_input = tf.cast(concat_input, tf.float32)
concat_input = np.transpose(np.squeeze(np.load(data_path)), [0,2,1])
print(concat_input.shape)
CRNN_saliency_map = generate_saliency_map(concat_input, CRNNCrispr_model)
CRNN_prediction = CRNNCrispr_model(concat_input)
print(CRNN_saliency_map.shape)
print(CRNN_prediction.shape)

In [ ]:
print(str(count_nonzero_grad(CRNN_saliency_map)) + " out of " + str(len(CRNN_saliency_map))
+ " samples in the saliency map of the sequencial inputs are nonzero")

In [ ]:
draw_saliency_map(saliency_map[index], 'Saliency Map of CRNNCrispr for max frequency')

# DeepCRISPR Saliency maps


In [ ]:
saliency_map_cls = np.load('saliency_values_class.npy')
saliency_map_cls.shape # (100, 1, 23, 8)

#100: saliency map for 100 different data points.
#1: single chanel
#23: length of sequencedat
#8: number of features A, C, G, T , CTCF, Dnase, H3k4me3, RRBS

saliency_map_cls = saliency_map_cls[:, :, :, :]  # Assuming single channel saliency map
print(saliency_map_cls.shape)

x_cls = np.load('x_input_class.npy')
x_cls = x_cls[:, :, :, :]  # Assuming single channel saliency map
print(x_cls.shape)

In [ ]:
sample_index = 0

x_cls_sqz = np.squeeze(x_cls[sample_index])
print(x_cls_sqz.shape)
saliency_map_cls_sqz = np.squeeze(saliency_map_cls[sample_index])
saliency_map_cls_sqz = saliency_map_cls_sqz.transpose()


# Plot the original input and its saliency map

plt.imshow(x_cls_sqz, cmap='gray')
plt.title('Original Input for DeepCRISPR Classification')
plt.show()

plt.imshow(saliency_map_cls_sqz)
plt.title('Saliency Map for DeepCRISPR Classification')
plt.show()

In [ ]:
cls_sqz = np.squeeze(saliency_map_cls, axis=1)  # Resulting shape: (100, 23, 8)
print(cls_sqz.shape)
cls_sum = np.sum(cls_sqz, axis=0).transpose() # Resulting shape: (23, 8)
print(cls_sum.shape)
heatmap = plt.get_cmap('viridis')(cls_sum)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(cls_sum, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for DeepCRISPR Classification')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()

### On-target Sequential Regression Task


In [ ]:
saliency_map_seq = np.load('saliency_values_seq.npy')
print(saliency_map_seq.shape)
#10: saliency map for 100 different data points.
#1: single chanel
#23: length of sequencedat
#4: number of features A, C, G, T , CTCF, Dnase, H3k4me3, RRBS

x_seq = np.load('x_input_seq.npy')
x_seq = x_seq[:, :, :, :]  # Assuming single channel saliency map
print(x_seq.shape)

In [ ]:
sample_index = 0

x_seq_sqz = np.squeeze(x_seq[sample_index])
print(x_seq_sqz.shape)
saliency_map_seq_sqz = np.squeeze(saliency_map_seq[sample_index])
print(saliency_map_seq_sqz.shape)
saliency_map_seq_sqz = saliency_map_seq_sqz.transpose()


# Plot the original input and its saliency map

plt.imshow(x_seq_sqz, cmap='gray')  # Assuming grayscale input
plt.title('Original Input for DeepCRISPR Sequential Regression ')
plt.show()

plt.imshow(saliency_map_seq_sqz)  # Assuming grayscale input
plt.title('Saliency Map for DeepCRISPR Sequential Regression')
plt.show()

In [ ]:
seq_sqz = np.squeeze(saliency_map_seq, axis=1)  # Resulting shape: (100, 23, 8)
seq_sum = np.sum(seq_sqz, axis=0).transpose() # Resulting shape: (23, 8)

heatmap = plt.get_cmap('viridis')(seq_sum)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(seq_sum, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for DeepCRISPR Sequential Regression Task')
y_ticks = np.arange(0, 4, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()

### On-target epigenetic regression task

In [ ]:
saliency_map_reg = np.load('saliency_values_reg_norm.npy')
saliency_map_reg.shape # (100, 1, 23 ,8)
#100: saliency map for 100 different data points.
#1: single chanel
#23: length of sequencedat
#8: number of features A, C, G, T , CTCF, Dnase, H3k4me3, RRBS

saliency_map_reg = saliency_map_reg[:, :, :, :]  # Assuming single channel saliency map
print(saliency_map_reg.shape)

x_reg = np.load('x_input_reg.npy')
x_reg = x_reg[:, :, :, :]  # Assuming single channel saliency map
print(x_reg.shape)

In [ ]:
sample_index = 0

x_reg_sqz = np.squeeze(x_reg[sample_index])
print(x_reg_sqz.shape)
saliency_map_reg_sqz = np.squeeze(saliency_map_reg[sample_index])
saliency_map_reg_sqz = saliency_map_reg_sqz.transpose()


# Plot the original input and its saliency map

plt.imshow(x_reg_sqz, cmap='gray')
plt.title('Original Input for DeepCRISPR Regression Task')
plt.show()

plt.imshow(saliency_map_reg_sqz)
plt.title('Saliency Map for DeepCRISPR Regression Task')
plt.show()

In [ ]:
y_input = np.load('y_input.npy')
predicted_on_target = np.load('predicted_on_target.npy')

In [ ]:
mean_value = np.mean(y_input)
mean_index = np.where(np.isclose(y_input, mean_value, atol=1e-3))[0]

print("mean value:", mean_value)
print("meen seq :", x_reg[mean_index])
print("mean index:", mean_index)

saliency_map_reg_mean_sqz = np.squeeze(saliency_map_reg[mean_index])
saliency_map_reg_mean_sqz = saliency_map_reg_mean_sqz.transpose()

heatmap = plt.get_cmap('viridis')(saliency_map_reg_mean_sqz)

plt.figure(figsize=(12, 6))
plt.imshow(saliency_map_reg_mean_sqz, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for DeepCRISPR of mean')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()

In [ ]:
min_index = np.argmin(y_input) # 16
print("min value:", y_input[min_index])
print("min seq :", x_reg[min_index])
print("min index:", min_index)
saliency_map_reg_min_sqz = np.squeeze(saliency_map_reg[min_index]) #17 #85
saliency_map_reg_min_sqz = saliency_map_reg_min_sqz.transpose()

heatmap = plt.get_cmap('viridis')(saliency_map_reg_min_sqz)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(saliency_map_reg_min_sqz, cmap='viridis')
plt.colorbar()

# plt.title('Saliency Map for Regression Task for max frequency')
plt.title('Saliency Map for DeepCRISPR for min frequency')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()

In [ ]:
max_index = np.argmax(y_input) # 16
print("max value:", y_input[max_index])
print("max seq :", x_reg[max_index])
print("max index:", max_index)


saliency_map_reg_max_sqz = np.squeeze(saliency_map_reg[max_index])
saliency_map_reg_max_sqz = saliency_map_reg_max_sqz.transpose()

heatmap = plt.get_cmap('viridis')(saliency_map_reg_max_sqz)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(saliency_map_reg_max_sqz, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for for DeepCRISPR of max frequency')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

padded_seq = np.pad(seq_2, ((0, 0), (0, 0), (0, 0), (0, num_zeros_to_pad)), mode='constant')
data = np.concatenate((padded_seq, saliency_map_reg[:-2]))

# data = saliency_map_reg[:-2]
num_datasets = data.shape[0]
print(num_datasets)
data_squeezed = np.squeeze(data, axis=1)  # Resulting shape: (100, 23, 8)
summed_data = np.sum(data_squeezed, axis=0)  # Resulting shape: (23, 8)

ft_importance = summed_data / num_datasets
ft_importance = ft_importance.transpose()


heatmap = plt.get_cmap('viridis')(ft_importance)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(ft_importance, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for DeepCRISPR Regression')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()


### On-target Reg Feature Importance Map

In [ ]:
# Maybe sum up all data within diff category?

cls_1 = saliency_map_cls
seq_2 = saliency_map_seq
reg_3 = saliency_map_reg

num_zeros_to_pad = 8 - seq_2.shape[-1]
print(num_zeros_to_pad)

padded_seq = np.pad(seq_2, ((0, 0), (0, 0), (0, 0), (0, num_zeros_to_pad)), mode='constant')
cls_reg = np.concatenate((cls_1, reg_3))

# Concatenate concatenated_array1 and array3 along the last axis
final_result = np.concatenate((cls_reg, padded_seq))
final_result.shape


In [ ]:
# data = np.concatenate((saliency_map_cls, saliency_map_reg))
# print(data.shape)
# num_datasets = data.shape[0]
# print(num_datasets)

data = final_result
num_datasets = data.shape[0]
print(num_datasets)
data_squeezed = np.squeeze(data, axis=1)  # Resulting shape: (100, 23, 8)
summed_data = np.sum(data_squeezed, axis=0)  # Resulting shape: (23, 8)

ft_importance = summed_data / num_datasets
ft_importance = ft_importance.transpose()


heatmap = plt.get_cmap('viridis')(ft_importance)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(ft_importance, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for DeepCRISPR for all tasks')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()


### Other 2 models comparison visualizatiton

In [ ]:
import pandas as pd

def grna_preprocess(lines):
    length = 23
    data_n = len(lines)
    seq = np.zeros((data_n, length, 4), dtype=int)
    for l in range(data_n):
        data = lines[l]
        seq_temp = data
        for i in range(length):
            if seq_temp[i] in "Aa":
                seq[l, i, 0] = 1
            elif seq_temp[i] in "Cc":
                seq[l, i, 1] = 1
            elif seq_temp[i] in "Gg":
                seq[l, i, 2] = 1
            elif seq_temp[i] in "Tt":
                seq[l, i, 3] = 1
    return seq


def epi_preprocess(lines):
    length = 23
    data_n = len(lines)
    epi = np.zeros((data_n, length), dtype=int)
    for l in range(data_n):
        data = lines[l]
        epi_temp = data
        for i in range(length):
            if epi_temp[i] in "A":
                epi[l, i] = 1
            elif epi_temp[i] in "N":
                epi[l, i] = 0
    return epi


def preprocess(file_path, usecols):
    data = pd.read_csv(file_path, usecols=usecols)
    data = np.array(data)
    ctcf, dnase, h3k4me3, rrbs = epi_preprocess(data[:, 0]), epi_preprocess(data[:, 1]), epi_preprocess(data[:, 2]), epi_preprocess(data[:, 3])
    epi = []
    for i in range(len(data)):
        ctcf_t, dnase_t, h3k4me3_t, rrbs_t = pd.DataFrame(ctcf[i]), pd.DataFrame(dnase[i]), pd.DataFrame(h3k4me3[i]), pd.DataFrame(rrbs[i])
        epi_t = pd.concat([ctcf_t, dnase_t, h3k4me3_t, rrbs_t], axis=1)
        epi_t = np.array(epi_t)
        epi.append(epi_t)
    epi = np.array(epi)
    return epi


def load_data(train_file):
    train_data = pd.read_csv(train_file, usecols=[4, 9])
    train_data = np.array(train_data)
    train_seq, train_y = train_data[:, 0], train_data[:, 1]
    train_seq = grna_preprocess(train_seq)
    train_epi = preprocess(train_file, [5, 6, 7, 8])
    train_y = train_y.reshape(len(train_y), -1)

    return train_seq, train_epi, train_y


In [ ]:
## Decode max/min sequence into binary array

# train_seq, train_epi, train_y = load_data("max_min_data.csv")
# seq_expand = train_seq.reshape((2, 4, 1, 23))
# epi_expand = train_seq.reshape((2, 4, 1, 23))
# merged_array = np.concatenate((seq_expand, epi_expand), axis=1)
# print(merged_array)
# print(train_y)
# np.save("merged_array.npy", merged_array)

In [ ]:
x_cls_1 = np.squeeze(x_cls)
x_seq_2 = np.squeeze(x_seq)
x_reg_3 = np.squeeze(x_reg)

num_zeros_to_pad = 4
print(num_zeros_to_pad)

x_padded_seq = np.pad(x_seq_2, ((0, 0), (0, 4), (0, 0)), mode='constant')
x_cls_reg = np.concatenate((x_cls_1, x_reg_3))

# Concatenate concatenated_array1 and array3 along the last axis
x_sum = np.concatenate((x_cls_reg, x_padded_seq))
x_sum.shape

In [ ]:
# np.save('x_sum.npy', x_sum)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

data = x_reg
actual_frequency = y_input
predicted_frequency = predicted_on_target

reshaped_data = data.reshape((100, -1))

model = LinearRegression()
model.fit(reshaped_data, actual_frequency)

feature_importance = model.coef_.reshape((8, 23))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_datasets, num_features, seq_length = x_sum.shape

# Function to generate a heatmap for nucleotides or dimers importance
def generate_heatmap(data, title, x_labels, y_labels):
    plt.figure(figsize=(12, 6))
    plt.imshow(data, cmap='viridis', interpolation='nearest')
    plt.title(title)
    plt.xlabel("Genomic Position")
    plt.ylabel("Nucleotides")
    plt.xticks(np.arange(seq_length), labels=x_labels)
    plt.yticks(np.arange(len(y_labels)), labels=y_labels)
    plt.colorbar()
    plt.show()



# Generate and display heatmaps
nucleotide_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']

generate_heatmap(feature_importance, "Feature Importance Heatmap for DeepCRISPR", np.arange(seq_length) + 1, nucleotide_labels)


In [ ]:
# x_test, epi_test, y_test = load_data('C-RNNCrispr_data.csv') # needs to upload
# for y_indv in y_input:
#   if y_indv in set(y_test.flatten()):
#     print(y_indv)
# print("terminated")

### Model Comparison Graph

In [ ]:
import matplotlib.pyplot as plt

# Data
methods = ['DeepCRISPR', 'CNN_SVR', 'CRNN']
spearman_values = [0.246, 0.7, 0.877]
auroc_values = [0.804, 0.94, 0.976]

# Plotting
fig, ax = plt.subplots()

ax.scatter(methods, spearman_values, label='Spearman', marker='o')

ax.scatter(methods, auroc_values, label='AUROC', marker='o')

# Adding labels and title
ax.set_ylabel('Performance Metrics')
ax.set_title('Performance Comparison of Methods')
ax.legend()

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#
methods = ['DeepCRISPR', 'CNN_SVR', 'CRNN']
spearman_values = [0.246, 0.7, 0.877]
auroc_values = [0.804, 0.94, 0.976]

bar_width = 0.35

index = np.arange(len(methods))

fig, ax = plt.subplots()

bar_spearman = ax.bar(index, spearman_values, bar_width, label='Spearman')

bar_auroc = ax.bar(index + bar_width, auroc_values, bar_width, label='AUROC')

ax.set_xlabel('Methods')
ax.set_ylabel('Performance Metrics')
ax.set_title('Performance Comparison of Methods')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(methods)
ax.legend()

for bar in bar_spearman + bar_auroc:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 3), ha='center', va='bottom')

plt.show()


# SVR Visualization

In [ ]:
# Maybe sum up all data within diff category?

cls_1 = saliency_map_cls
seq_2 = saliency_map_seq
reg_3 = saliency_map_reg

num_zeros_to_pad = 8 - seq_2.shape[-1]
print(num_zeros_to_pad)

padded_seq = np.pad(seq_2, ((0, 0), (0, 0), (0, 0), (0, num_zeros_to_pad)), mode='constant')
cls_reg = np.concatenate((cls_1, reg_3))

# Concatenate concatenated_array1 and array3 along the last axis
final_result = np.concatenate((cls_reg, padded_seq))
final_result.shape


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# data = np.concatenate((saliency_map_cls, saliency_map_reg))
# print(data.shape)
# num_datasets = data.shape[0]
# print(num_datasets)

data = final_result
num_datasets = data.shape[0]
print(num_datasets)
data_squeezed = np.squeeze(data, axis=1)  # Resulting shape: (100, 23, 8)
summed_data = np.sum(data_squeezed, axis=0)  # Resulting shape: (23, 8)

ft_importance = summed_data / num_datasets
ft_importance = ft_importance.transpose()


heatmap = plt.get_cmap('viridis')(ft_importance)

# Display the original image, saliency map, and overlaid image
plt.figure(figsize=(12, 6))
plt.imshow(ft_importance, cmap='viridis')
plt.colorbar()

plt.title('Saliency Map for DeepCRISPR for all tasks')
y_ticks = np.arange(0, 8, 1)  # Custom tick locations
y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

x_ticks = np.arange(0, 23, 1)  # Custom tick locations
x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

plt.xlabel('Genomic Position')
plt.ylabel('Nucleotide')

plt.show()


PCA on the output from CNN (2D)

In [ ]:
from sklearn.decomposition import PCA

pca_2 = PCA(n_components=2)

print(x_train.shape)
compressed_train = pca_2.fit_transform(x_train)
print(compressed_train.shape)


In [ ]:
x1 = compressed_train[:,0]
x2 = compressed_train[:,1]

# Create a scatter plot with colors based on values
plt.scatter(x1, x2, c=y_train, cmap='viridis', marker='o', s=20)

# Add a colorbar for reference
plt.colorbar(label='Values')

# Set labels and title
plt.xlabel('Principle Component 1')
plt.ylabel('Principle Component 2')
plt.title('PCA of CNN-SVR gound truth')

# Show the plot
plt.show()

In [ ]:
x1 = compressed_train[:,0]
x2 = compressed_train[:,1]

# Create a scatter plot with colors based on values
plt.scatter(x1, x2, c=y_pred_train, cmap='viridis', marker='o', s=20)

# Add a colorbar for reference
plt.colorbar(label='Values')

# Set labels and title
plt.xlabel('Principle Component 1')
plt.ylabel('Principle Component 2')
plt.title('PCA of CNN-SVR prediction')

# Show the plot
plt.show()

PCA on the output from CNN (1D)

In [ ]:
pca_1 = PCA(n_components=1)

print(x_train.shape)
compressed_train_1d = pca_1.fit_transform(x_train)
print(compressed_train_1d.shape)

print(x_test.shape)
compressed_test_1d = pca_1.fit_transform(x_test)
print(compressed_test_1d.shape)


In [ ]:
# Plot the results
plt.scatter(compressed_train_1d, y_train, color='darkorange', marker='o', s=20, label='data')
plt.plot(compressed_train_1d, y_pred_train, color='navy', lw=2, label='prediction training')
plt.plot(compressed_test_1d, y_pred, color='green', lw=2, label='prediction testing')

plt.xlabel('Principle Component 1')
plt.ylabel('Values')
plt.title('SVR Visualization 1D')
plt.legend()
plt.show()

# CRNNCrispr Class Optimization

In [ ]:
def CRNN_input_optimization_analysis(model, target_output, input_shape, num_iterations=100):
    np.random.seed(41)
    seq = tf.Variable(np.random.random(input_shape), dtype=tf.float32)
    np.random.seed(42)
    epi = tf.Variable(np.random.random(input_shape), dtype=tf.float32)
    optimizer = tf.optimizers.Adam(learning_rate=0.1)

    for _ in range(num_iterations):
        with tf.GradientTape() as tape:
            output = model([tf.expand_dims(seq, axis=0), tf.expand_dims(epi, axis=0)])
            loss = tf.abs(output - target_output)
        gradients_seq, gradients_epi = tape.gradient(loss, [seq, epi])
        optimizer.apply_gradients(zip([gradients_seq, gradients_epi], [seq, epi]))

    optimized_seq = seq.numpy()
    optimized_epi = epi.numpy()
    optimized_output = model.predict([np.expand_dims(optimized_seq, axis=0), np.expand_dims(optimized_epi, axis=0)])[0][0]

    return optimized_seq, optimized_epi, optimized_output

In [ ]:
optimized_seq, optimized_epi, optimized_output = CRNN_input_optimization_analysis(CRNNCrispr_model, 1, (23, 4), 100)
print(optimized_seq.shape)
print(optimized_epi.shape)
print(optimized_output)

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
np.random.seed(41)
plt.imshow(np.random.random((23, 4)))
plt.title('Original Random Sequencial Input')
plt.axis('off')

plt.subplot(2, 2, 3)
np.random.seed(42)
plt.imshow(np.random.random((23, 4)))
plt.title('Original Random Epigenetic Input')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(optimized_seq)
plt.title('Optimized Sequencial Input')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(optimized_epi)
plt.title('Optimized Epigenetic Input')
plt.axis('off')

plt.show()

import numpy as np

def decode_dna(one_hot_sequence):
    base_mapping = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}
    indices = np.argmax(one_hot_sequence, axis=1)
    decoded_sequence = ''.join([base_mapping[idx] for idx in indices])
    return decoded_sequence

print(decode_dna(optimized_seq))
print(decode_dna(optimized_epi))

# SHAP VALUE

In [ ]:
!pip install deeplift
!pip install shap
from deeplift.visualization import viz_sequence
import shap

In [ ]:
print("SHAP version is:", shap.__version__)
print("Tensorflow version is:", tf.__version__)

In [ ]:
# Data for min and max
deepcrispr_data_dir = "My Drive/CSCI 2952G Final Project/x_input_reg.npy"
deepcrispr_path = f"{drive_dir}/{deepcrispr_data_dir}"

concat_input = np.transpose(np.squeeze(np.load(deepcrispr_path)), [0,2,1])
print(concat_input.shape)

In [ ]:
# Calculate the mean of epigenetic tracks at each position
mean_input = tf.reduce_mean(concat_input, axis=0)
mean_seq, mean_epi = tf.split(mean_input, 2, axis=1)
print(mean_seq)
print(mean_epi)

In [ ]:
# Defining different reference values

# zero input
reference_zeroes = tf.zeros(input_test[0].shape)
reference_zeroes = np.ravel(reference_zeroes)
reference_zeroes = np.reshape(reference_zeroes, (1, reference_zeroes.shape[0]))

print("reference zero input shape: " + str(reference_zeroes.shape))

In [ ]:
# expected frequency
expected_fre = tf.constant([0.3, 0.2, 0.2, 0.3])
expected_fre = tf.reshape(expected_fre, (1, expected_fre.shape[0]))
multiples = tf.constant([23, 1])
reference_expected_seq = tf.tile(expected_fre, multiples)

reference_expected_epi = mean_epi

reference_expected = tf.concat([reference_expected_seq, reference_expected_epi], axis=1)

reference_expected = np.ravel(reference_expected)
reference_expected = np.reshape(reference_expected, (1, reference_expected.shape[0]))
print("reference expected input shape: " + str(reference_expected.shape))

In [ ]:
max_index = 19
min_index = 16
max_input = concat_input[max_index]
min_input = concat_input[min_index]
max_input = np.reshape(max_input, (1, 23, 8))
min_input = np.reshape(min_input, (1, 23, 8))

print("max data shape: " + str(max_input.shape))
print("min data shape: " + str(min_input.shape))

In [ ]:
def draw_shap_map(saliency_map, title_name):
    # Normalize the saliency map
    saliency_map /= (tf.reduce_max(tf.abs(saliency_map)) + 1e-8)

    # Transpose the map to match the format in the paper
    saliency_map = np.transpose(saliency_map)

    # Create a heatmap by applying a colormap (e.g., 'viridis')
    heatmap = plt.get_cmap('viridis')(saliency_map)

    # Display the original image, saliency map, and overlaid image
    plt.figure(figsize=(12, 6))
    plt.imshow(saliency_map, cmap='viridis')
    plt.colorbar()

    plt.title(title_name)
    y_ticks = np.arange(0, 8, 1)  # Custom tick locations
    y_tick_labels = ['A', 'C', 'G', 'T', 'CTCF', 'Dnase', 'H3k4me3', 'RRBS']  # Custom tick labels

    x_ticks = np.arange(0, 23, 1)  # Custom tick locations
    x_tick_labels = np.arange(1, 24, 1)  # Custom tick labels

    plt.xticks(x_ticks, x_tick_labels)
    plt.yticks(y_ticks, y_tick_labels)

    plt.xlabel('Genomic Position')
    plt.ylabel('Nucleotide')

    plt.show()

### CNNSVR

In [ ]:
max_embed = model.predict(max_input)[:, selected_cnn_fea_cols]
min_embed = model.predict(min_input)[:, selected_cnn_fea_cols]

max_pred = clf.predict(max_embed)
min_pred = clf.predict(min_embed)

print("max data model prediction: " + str(max_pred))
print("min data model prediction: " + str(min_pred))


max_data = np.ravel(max_input)
max_data = np.reshape(max_data, (1, max_data.shape[0]))
print("max data shape: " + str(max_data.shape))

min_data = np.ravel(min_input)
min_data = np.reshape(min_data, (1, min_data.shape[0]))
print("min data shape: " + str(min_data.shape))

def f(X):
  num_sample = X.shape[0]
  input = np.reshape(X, (num_sample, 23, 8))
  embed = model.predict(input)[:, selected_cnn_fea_cols]
  return clf.predict(embed)

Explainer using reference zeroes

In [ ]:
# Explainer using reference_zeros
explainer = shap.KernelExplainer(f, reference_zeroes)
print("expected importance for the reference value: " + str(explainer.expected_value))

shap_values_max = explainer.shap_values(max_data)
shap_values_min = explainer.shap_values(min_data)

In [ ]:
shap.initjs()

shap_values_max = np.reshape(shap_values_max, (23,8))
seq_max, epi_max = np.split(shap_values_max, 2, axis = 1)

shap_values_min = np.reshape(shap_values_min, (23,8))
seq_min, epi_min = np.split(shap_values_min, 2, axis = 1)

# project the importance at each position onto the base that's actually present
seq_input_max, epi_input_max = np.split(max_input[0], 2, axis = 1)
seq_input_min, epi_input_min = np.split(min_input[0], 2, axis = 1)

print(seq_input_max)
print(seq_input_min)


processed_max = (
    np.sum(seq_max, axis=-1)[:, None] * seq_input_max
)

processed_min = (
    np.sum(seq_min, axis=-1)[:, None] * seq_input_min
)
print(processed_max)
print(processed_min)

viz_sequence.plot_weights(processed_max, subticks_frequency=1)
viz_sequence.plot_weights(processed_min, subticks_frequency=1)

In [ ]:
draw_shap_map(shap_values_max, 'SHAP Value for max')

draw_shap_map(shap_values_min, 'SHAP Value for min')

Explainer using reference expected

In [ ]:
# Explainer using reference_expected
explainer = shap.KernelExplainer(f, reference_expected)
print("expected importance for the reference value: " + str(explainer.expected_value))

shap_values_max = explainer.shap_values(max_data)
shap_values_min = explainer.shap_values(min_data)

In [ ]:
shap.initjs()

shap_values_max = np.reshape(shap_values_max, (23,8))
seq_max, epi_max = np.split(shap_values_max, 2, axis = 1)

shap_values_min = np.reshape(shap_values_min, (23,8))
seq_min, epi_min = np.split(shap_values_min, 2, axis = 1)

# project the importance at each position onto the base that's actually present
seq_input_max, epi_input_max = np.split(max_input[0], 2, axis = 1)
seq_input_min, epi_input_min = np.split(min_input[0], 2, axis = 1)

print(seq_input_max)
print(seq_input_min)


processed_max = (
    np.sum(seq_max, axis=-1)[:, None] * seq_input_max
)

processed_min = (
    np.sum(seq_min, axis=-1)[:, None] * seq_input_min
)
print(processed_max)
print(processed_min)

viz_sequence.plot_weights(processed_max, subticks_frequency=1)
viz_sequence.plot_weights(processed_min, subticks_frequency=1)

In [ ]:
draw_shap_map(shap_values_max, 'SHAP Value for max')

draw_shap_map(shap_values_min, 'SHAP Value for min')

### CRNNCrispr

In [ ]:
max_pred = CRNNCrispr_model.predict(max_input)
min_pred = CRNNCrispr_model.predict(min_input)

print("max data model prediction: " + str(max_pred))
print("min data model prediction: " + str(min_pred))


max_data = np.ravel(max_input)
max_data = np.reshape(max_data, (1, max_data.shape[0]))
print("max data shape: " + str(max_data.shape))

min_data = np.ravel(min_input)
min_data = np.reshape(min_data, (1, min_data.shape[0]))
print("min data shape: " + str(min_data.shape))

def f(X):
  num_sample = X.shape[0]
  input = np.reshape(X, (num_sample, 23, 8))
  return CRNNCrispr_model.predict(input)

Explainer using reference zeroes

In [ ]:
explainer = shap.KernelExplainer(f, reference_zeroes)
print("expected importance for the reference value: " + str(explainer.expected_value))

shap_values_max = explainer.shap_values(max_data)
shap_values_min = explainer.shap_values(min_data)

In [ ]:
shap.initjs()

shap_values_max = np.reshape(shap_values_max, (23,8))
seq_max, epi_max = np.split(shap_values_max, 2, axis = 1)

shap_values_min = np.reshape(shap_values_min, (23,8))
seq_min, epi_min = np.split(shap_values_min, 2, axis = 1)

# project the importance at each position onto the base that's actually present

# print(seq_max)
# print(seq[0])


seq_input_max, epi_input_max = np.split(max_input[0], 2, axis = 1)
seq_input_min, epi_input_min = np.split(min_input[0], 2, axis = 1)

print(seq_input_max)
print(seq_input_min)


processed_max = (
    np.sum(seq_max, axis=-1)[:, None] * seq_input_max
)

processed_min = (
    np.sum(seq_min, axis=-1)[:, None] * seq_input_min
)
print(processed_max)
print(processed_min)

viz_sequence.plot_weights(processed_max, subticks_frequency=1)
viz_sequence.plot_weights(processed_min, subticks_frequency=1)

In [ ]:
draw_shap_map(shap_values_max, 'SHAP Value for max')

draw_shap_map(shap_values_min, 'SHAP Value for min')

Explainer using reference expected

In [ ]:
expected_explainer = shap.KernelExplainer(f, reference_expected)
print("expected importance for the reference value: " + str(expected_explainer.expected_value))

shap_values_max = explainer.shap_values(max_data)
shap_values_min = explainer.shap_values(min_data)

In [ ]:
shap.initjs()

shap_values_max = np.reshape(shap_values_max, (23,8))
seq_max, epi_max = np.split(shap_values_max, 2, axis = 1)

shap_values_min = np.reshape(shap_values_min, (23,8))
seq_min, epi_min = np.split(shap_values_min, 2, axis = 1)

# project the importance at each position onto the base that's actually present

# print(seq_max)
# print(seq[0])


seq_input_max, epi_input_max = np.split(max_input[0], 2, axis = 1)
seq_input_min, epi_input_min = np.split(min_input[0], 2, axis = 1)

print(seq_input_max)
print(seq_input_min)


processed_max = (
    np.sum(seq_max, axis=-1)[:, None] * seq_input_max
)

processed_min = (
    np.sum(seq_min, axis=-1)[:, None] * seq_input_min
)
print(processed_max)
print(processed_min)

viz_sequence.plot_weights(processed_max, subticks_frequency=1)
viz_sequence.plot_weights(processed_min, subticks_frequency=1)

In [ ]:
draw_shap_map(shap_values_max, 'SHAP Value for max')

draw_shap_map(shap_values_min, 'SHAP Value for min')

### SHAP Examples with CRNNCrispr


In [ ]:
%matplotlib inline
from __future__ import print_function, division

In [ ]:
! [[ ! -f sequences.simdata.gz ]] && wget https://raw.githubusercontent.com/AvantiShri/model_storage/db919b12f750e5844402153233249bb3d24e9e9a/deeplift/genomics/sequences.simdata.gz
! [[ ! -f keras2_conv1d_record_5_model_PQzyq_modelJson.json ]] && wget https://raw.githubusercontent.com/AvantiShri/model_storage/b6e1d69/deeplift/genomics/keras2_conv1d_record_5_model_PQzyq_modelJson.json
! [[ ! -f keras2_conv1d_record_5_model_PQzyq_modelWeights.h5 ]] && wget https://raw.githubusercontent.com/AvantiShri/model_storage/b6e1d69/deeplift/genomics/keras2_conv1d_record_5_model_PQzyq_modelWeights.h5
! [[ ! -f test.txt.gz ]] && wget https://raw.githubusercontent.com/AvantiShri/model_storage/9aadb769735c60eb90f7d3d896632ac749a1bdd2/deeplift/genomics/test.txt.gz

In [ ]:
! pip install simdna
! pip install shap

In [ ]:
import gzip

import simdna.synthetic as synthetic

data_filename = "sequences.simdata.gz"

# read in the data in the testing set
test_ids_fh = gzip.open("test.txt.gz", "rb")
ids_to_load = [x.decode("utf-8").rstrip("\n") for x in test_ids_fh]
data = synthetic.read_simdata_file(data_filename, ids_to_load=ids_to_load)

In [ ]:
import numpy as np


# this is set up for 1d convolutions where examples
# have dimensions (len, num_channels)
# the channel axis is the axis for one-hot encoding.
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence), 4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(
        zeros_array=to_return, sequence=sequence, one_hot_axis=1
    )
    return to_return


def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis == 0 or one_hot_axis == 1
    if one_hot_axis == 0:
        assert zeros_array.shape[1] == len(sequence)
    elif one_hot_axis == 1:
        assert zeros_array.shape[0] == len(sequence)
    # will mutate zeros_array
    for i, char in enumerate(sequence):
        if char == "A" or char == "a":
            char_idx = 0
        elif char == "C" or char == "c":
            char_idx = 1
        elif char == "G" or char == "g":
            char_idx = 2
        elif char == "T" or char == "t":
            char_idx = 3
        elif char == "N" or char == "n":
            continue  # leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: " + str(char))
        if one_hot_axis == 0:
            zeros_array[char_idx, i] = 1
        elif one_hot_axis == 1:
            zeros_array[i, char_idx] = 1


onehot_data = np.array(
    [one_hot_encode_along_channel_axis(seq) for seq in data.sequences]
)

In [ ]:
from keras.models import model_from_json

# load the keras model
keras_model_weights = "keras2_conv1d_record_5_model_PQzyq_modelWeights.h5"
keras_model_json = "keras2_conv1d_record_5_model_PQzyq_modelJson.json"

keras_model = model_from_json(open(keras_model_json).read())
keras_model.load_weights(keras_model_weights)

In [ ]:
pip install --upgrade deeplift==0.6.10.0

In [ ]:
import deeplift
import shap
print("deeplift version is:", deeplift.__version__)
print("shap version is:", shap.__version__)

In [ ]:
from deeplift.dinuc_shuffle import (
    prepare_edges,
    shuffle_edges,
    traverse_edges,
)


def onehot_dinuc_shuffle(s):
    s = np.squeeze(s)
    argmax_vals = "".join([str(x) for x in np.argmax(s, axis=-1)])
    shuffled_argmax_vals = [
        int(x)
        for x in traverse_edges(argmax_vals, shuffle_edges(prepare_edges(argmax_vals)))
    ]
    to_return = np.zeros_like(s)
    to_return[list(range(len(s))), shuffled_argmax_vals] = 1
    return to_return


def shuffle_several_times(s):
    return np.array([onehot_dinuc_shuffle(s) for i in range(100)])

In [ ]:
import numpy as np
from deeplift.visualization import viz_sequence

import shap
import shap.explainers._deep.deep_tf

np.random.seed(1)
seqs_to_explain = onehot_data[[0, 3, 9]]  # these three are positive for task 0

# print(seqs_to_explain.shape)
# reference_val = np.zeros(seqs_to_explain[0].shape)
# reference_val = np.reshape(reference_val, (1, 200, 4))


dinuc_shuff_explainer = shap.DeepExplainer(
    (keras_model.input, keras_model.output[:, 0]), shuffle_several_times
)
print(dinuc_shuff_explainer.expected_value)


raw_shap_explanations = dinuc_shuff_explainer.shap_values(seqs_to_explain)

# project the importance at each position onto the base that's actually present
dinuc_shuff_explanations = (
    np.sum(raw_shap_explanations, axis=-1)[:, :, None] * seqs_to_explain
)
for dinuc_shuff_explanation in dinuc_shuff_explanations:
    viz_sequence.plot_weights(dinuc_shuff_explanation, subticks_frequency=20)